In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

# Set path to find modelling tools for later use
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))


from haberrspd.preprocess import preprocessMJFF
                         
import pandas as pd
import re
import numpy as np
print(np.__version__)
from collections import Counter, defaultdict
import itertools
from operator import itemgetter
from scipy.stats import (gamma, lognorm, gengamma)

from pathlib import Path
from IPython.display import display

# Plot stuff
import seaborn as sns
from scipy.constants import golden
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Depending on where I am, set the path
import socket
if socket.gethostname() == 'pax':
    # Monster machine
    data_root = '../data/MJFF/' # My local path
    data_root = Path(data_root)
else:
    # Laptop
    data_root = '/home/nd/data/liverpool/MJFF' # My local path
    data_root = Path(data_root)

1.16.4


----

# Character + Timing data

In [ ]:
proc = preprocessMJFF()

In [ ]:
out = proc('all')

In [ ]:
out.to_csv("../data/MJFF/preproc/EnglishSpanishData-preprocessed.csv",index=False)

In [ ]:
out = proc('spanish')

In [ ]:
out.to_csv("../data/MJFF/preproc/SpanishData-preprocessed.csv",index=False)

In [ ]:
out = proc('english')

In [ ]:
out.to_csv("../data/MJFF/preproc/EnglishData-preprocessed.csv",index=False)

In [ ]:
pwd

In [ ]:
out[out['Preprocessed_typed_sentence'].apply(lambda x: len(x) > 10000)]

In [ ]:
# df = pd.read_csv(data_root / 'SpanishData-duplicateeventsremoved.csv')
df = pd.read_csv(data_root / 'SpanishData.csv')
df_meta = pd.read_csv(data_root / "SpanishParticipantKey.csv",
                      index_col=0,
                      header=0,
                      names=['participant_id', 'diagnosis'])

# Character ONLY data

In [ ]:
proc = preprocessMJFF()

In [ ]:
out = proc('english',include_time=False)

In [ ]:
out.head()

In [ ]:
out[out['Preprocessed_typed_sentence'].apply(lambda x: len(x) < 30)]

In [ ]:
out.to_csv("../data/MJFF/preproc/char/EnglishSpanishData-preprocessed.csv",index=False)

## PLOT


In [ ]:
controls = set(df_meta.loc[df_meta.diagnosis == 0].participant_id)
pd_subjects = set(df_meta.loc[df_meta.diagnosis == 1].participant_id)

In [ ]:
pd_timestamp_diffs = []
# Loop over all subjects
for sub in pd_subjects:
    # Get all delta timestamps for this sentence, across all subjects
    pd_timestamp_diffs.extend(df.loc[(df.sentence_id == 57) & (df.participant_id == sub)].timestamp.diff().values)
    
control_timestamp_diffs = []
# Loop over all subjects
for sub in controls:
    # Get all delta timestamps for this sentence, across all subjects
    control_timestamp_diffs.extend(df.loc[(df.sentence_id == 57) & (df.participant_id == sub)].timestamp.diff().values)

In [ ]:
# remove NaNs
pd_cleaned_list = [x for x in pd_timestamp_diffs if str(x) != 'nan']
control_cleaned_list = [x for x in control_timestamp_diffs if str(x) != 'nan']

In [ ]:
# PD

# fixed bin size
bins = np.arange(0, 10000, 50) # fixed bin size
# plt.xlim([min(data)-5, max(data)+5])
fig = plt.figure(figsize=(14,6))
plt.hist(pd_cleaned_list, bins=bins, alpha=0.5)
# plt.vlines(np.quantile(cleaned_list,0.95),0,900,'r')
# plt.vlines(np.mean(cleaned_list),0,900,'b')
plt.grid(True)
plt.show()

In [ ]:
# control

# fixed bin size
bins = np.arange(0, 10000, 50) # fixed bin size
# plt.xlim([min(data)-5, max(data)+5])
fig = plt.figure(figsize=(14,6))
plt.hist(control_cleaned_list, bins=bins, alpha=0.5)
# plt.vlines(np.quantile(cleaned_list,0.95),0,900,'r')
# plt.vlines(np.mean(cleaned_list),0,900,'b')
plt.grid(True)
plt.show()

### Mechanical turk data

In [ ]:
out.to_csv(data_root /'preprocessed_MechanicalTurkCombinedEnglishData.csv',index=False)

In [ ]:
ls ../data/MJFF/

### IKI extraction

In [ ]:
out = create_mjff_iki_training_data(df)

## Keyboard inputs

In [ ]:
from haberrspd.charCNN.data_utils_tf import create_mjff_data_objects, us_standard_layout_keyboard, english_keys_to_2d_coordinates
import keras.backend as K
from keras import callbacks
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from numpy import array, int64, ones, hstack, pad, einsum, dstack
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow import cast, float32, one_hot
import itertools

In [ ]:
which_information = "char_time_space"
DATA_ROOT = Path("../data/") / "MJFF" / "preproc"
data_string = "EnglishData-preprocessed.csv"
if which_information == "char_time_space":
    # Get relevant long-format data
    which_information = "char_time"

In [ ]:
df = read_csv(DATA_ROOT / which_information / data_string, header=0)  # MJFF data
subject_documents, subjects_diagnoses, alphabet = create_mjff_data_objects(df)

# Store alphabet size
alphabet_size = len(alphabet)

print('Total number of characters:', alphabet_size)
alphabet_indices = dict((c, i) for i, c in enumerate(alphabet))

if which_information == "char_time" or which_information == "char_time_space":
    # Rounds (up) to nearest thousand
    max_sentence_length = round(df.Preprocessed_typed_sentence.apply(lambda x: len(x)).max(), -3)
if which_information == "char":
    # Rounds (up) to nearest hundred
    max_sentence_length = round(df.Preprocessed_typed_sentence.apply(lambda x: len(x)).max(), -2)

# Make training data array
all_sentences = [item for sublist in subject_documents for item in sublist]

In [ ]:
# Initialise tokenizer which maps characters to integers
tk = Tokenizer(num_words=None, char_level=True)

# Fit to text: convert all chars to ints
tk.fit_on_texts(all_sentences)

# Update alphabet
tk.word_index = alphabet_indices

# Get integer sequences: converts sequences of chars to sequences of ints
int_sequences = tk.texts_to_sequences(all_sentences)

# Pad sequences so that they all have the same length and then one-hot encode
X = to_categorical(pad_sequences(int_sequences, maxlen=max_sentence_length, padding='post'))

In [ ]:
X.shape

In [ ]:
which_information  =  'char_time_space'
if which_information == 'char_time_space':
    # Load relevant keyboard
    keyboard = us_standard_layout_keyboard()  # OBS: nested list
    # Check that all chars are in fact in our "keyboard" -- if not, we cannot map a coordinate
    assert alphabet.issubset(set(list(itertools.chain.from_iterable(keyboard))))
    space = [english_keys_to_2d_coordinates(sentence, keyboard) for sentence in all_sentences]
    space_padded = [pad(s, [(0, max_sentence_length - len(s)), (0, 0)], mode='constant') for s in space]
    # Append coordinates to one-hot encoded sentences
    X = einsum('ijk->kij', dstack([hstack((x, s)) for (x, s) in zip(X, space_padded)]))

In [ ]:
X.shape

# Document (participant) -level classification

Three options (at time of writing):

1. Submit each sentence to model and extract classification probability for each sentence, agglomorate at the end, and the conduct a classification on the vector of all 15 probabilities.
2. Calculate the expected value of all encoded (15) sentences and then pass this to the model and take the classification.
3. Vertically stack all embedded sentences, and let the convolution run over this (very long) array.

In [ ]:
from haberrspd.charCNN.data_utils_tf import create_training_data_keras, create_mjff_data_objects
from pandas import read_csv

In [ ]:
which_information = "char_time"
DATA_ROOT = Path("../data/") / "MJFF" / "preproc"
data_string = "EnglishData-preprocessed.csv"
df = read_csv(DATA_ROOT / which_information / data_string, header=0)  # MJFF data
# subject_documents, subjects_diagnoses, alphabet = create_mjff_data_objects(df)
# X_train, X_test, y_train, y_test, max_sentence_length, alphabet_size = create_training_data_keras(DATA_ROOT, which_information, data_string)

In [ ]:
df.head()

In [ ]:
for doc in subject_documents[:3]:
    print(doc[:2])
    print('\n')

# MRC data

0. Homogenise so that Spacebar is a blank character
0. Delete rubbish characters (i.e. remove the rows)
2. What to do with stuff like Shift
3. Figure out what to do when multiple characters are depressed simultaneously
4. Make lowercase all characters

In [2]:
from pandas import read_csv
from haberrspd.preprocess import clean_MRC, backspace_corrector, make_character_compression_time_sentence, reorder_key_timestamp_columns_mrc, calculate_edit_distance_between_response_and_target_MRC
from haberrspd.charCNN.data_utils_tf import us_english_keyboard_mrc
from numpy import concatenate
from typing import Tuple
import random
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

from itertools import compress, count
from operator import itemgetter

# Monster machine
data_root = '../data/MRC/' # My local path
data_root = Path(data_root)


Using TensorFlow backend.


In [3]:
df = read_csv(data_root / "CombinedTypingDataSept27.csv", header=0)

/home/neil/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Protocol to process the MRC data

1. A form of `create_char_compression_time_mjff_data` [to get the temporal data] <-- this needs to be set so that we get a list of 
2. Second use `create_dataframe_from_processed_data`

In [33]:
def create_char_compression_time_mrc_data(df: pd.DataFrame, 
                                          time_redux_fact=10) -> Tuple[dict, list]:

    assert set(["participant_id", "key", "timestamp", "sentence_id"]).issubset(df.columns)

    # All sentences will be stored here, indexed by their type
    char_compression_sentences = defaultdict(dict)
    
    # Get the unique number of subjects
    subjects = sorted(set(df.participant_id))

    # Loop over subjects
    for subj_idx in subjects:
        # Not all subjects have typed all sentences hence we have to do it this way
        for sent_idx in df.loc[(df.participant_id == subj_idx)].sentence_id.unique():
            
            # Locate df segment to extract
            coordinates = (df.participant_id == subj_idx) & (df.sentence_id == sent_idx)
            
            print("Participant: {}, Sentence: {}".format(subj_idx, sent_idx))
            
            # Get correctly ordered sentences and total compression times
            df_tmp = reorder_key_timestamp_columns_mrc(df.loc[coordinates, ('key','timestamp')])
            
            # TODO: check to see if the key-compression column follows the correct order
            
            
            # "correct" the sentence by operating on user backspaces 
            corrected_char_sentence, removed_chars_indx = backspace_corrector(df_tmp.key.tolist())
            
            compression_times = calculate_total_key_compression_time(df_tmp.drop(df_tmp.index[removed_chars_indx]))
            
            assert len(compression_times) == len(corrected_char_sentence[::2]), "Error at ({},{}).".format(subj_idx,sent_idx)
            assert any(x < 0 for x in compression_times) is False, "Error at ({},{}).".format(subj_idx,sent_idx) # Check no negative timings

            # Make long-format version of each typed, corrected, sentence
            # Note that we remove the last character to make the calculation correct.
            char_compression_sentences[subj_idx][sent_idx] = make_character_compression_time_sentence(compression_times,
                                                                                                      corrected_char_sentence[::2], 
                                                                                                      time_redux_fact)
    return char_compression_sentences

In [206]:
def lookup(v, d={}, c=count()):
    if v in d:
        return d.pop(v)
    else:
        d[v] = next(c)
        return d[v]

def reorder_key_timestamp_columns_mrc(df: pd.DataFrame):

    # Check that the column is of even length
    assert len(df) % 2 == 0, "The length is {}.".format(len(df))

    # Use lookup function to extract the next row-order
    df["new_row_order"] = df.key.map(lookup)
    
    # Don't return an object, just leave as is
    return df.sort_values(by="new_row_order", kind="mergesort").drop("new_row_order", axis=1).reset_index(drop=True)

In [515]:
# TODO: backstop removal before any of this is run properly
# Test just a single sentence here


fail=0
success=0

for subj_idx in random.choices(df.participant_id.unique().tolist(), k=10):
    # Not all subjects have typed all sentences hence we have to do it this way
    for sent_idx in df.loc[(df.participant_id == subj_idx)].sentence_id.unique():
        
        # Locate df segment to extract
        coordinates = (df.participant_id == subj_idx) & (df.sentence_id == sent_idx)
        
        # Store temporary dataframe because why not
        tmp = df.loc[coordinates, ('key','timestamp','type')].reset_index(drop=True) # Reset index 
        
        # Get correctly ordered sentences and total compression times
        new = new_sort(tmp)
        
        if not assess_repeating_key_compression_pattern(new.type.tolist()):
            # Condition fails: data-collection is broken

            combine_shift_keydowns_without_matching_keyup(new) 
            new.reset_index(drop=True, inplace=True) # Reset index here

            if assess_repeating_key_compression_pattern(new.type.tolist()):
                # Condition fails: data-collection is broken
                success+=1
            else:
                # Condition succeeds: data-collection is fixed
                
                fail+=1
                print("[broken sentence] Participant: {}, Sentence: {}".format(subj_idx, sent_idx))
        else:
            # Condition succeeds: data-collection is fixed
            
            success+=1
            
print(success, fail)

[broken sentence] Participant: 1025, Sentence: 11
[broken sentence] Participant: 1025, Sentence: 12
[broken sentence] Participant: 1038, Sentence: 11
[broken sentence] Participant: 33, Sentence: 2
110 4


In [469]:
# pd.DataFrame(data=np.hstack([tmp.values,new.values]),columns=['key','timestamp','type']*2).head(20)

In [575]:
### Locate df segment to extract
coordinates = (df.participant_id == 33) & (df.sentence_id == 120)
# Store temporary dataframe because why not
tmp = df.loc[coordinates, ('key','timestamp','type')]#.reset_index(drop=True) # Reset index 

tmp.timestamp.diff().sum()

0.0

In [679]:
for i in df.participant_id.unique():
    ss = df.loc[df.participant_id == i].timeStamp.diff().sum()
    if ss <= 0.0:
        print(i)

In [545]:

combine_shift_keydowns_without_matching_keyup(tmp) 
tmp.reset_index(drop=True, inplace=True) # Reset index here
# Get correctly ordered sentences and total compression times
new = new_sort(tmp)


In [648]:
d1 = ['bob']*2+['shift']*4 + ['shift'] + ["a"]*2
d2 = ['keydown', 'keyup'] +['keydown']*4 + ['keyup']+ ['keydown', 'keyup']
test_df = pd.DataFrame(data=np.array([d1,d2]).T, columns=['key','type'])
test_df

,key,type
0,bob,keydown
1,bob,keyup
2,shift,keydown
3,shift,keydown
4,shift,keydown
5,shift,keydown
6,shift,keyup
7,a,keydown
8,a,keyup


In [658]:
combine_shift_keydowns_without_matching_keyup(test_df)
test_df

,key,type
0,bob,keydown
1,bob,keyup
5,shift,keydown
6,shift,keyup
7,a,keydown
8,a,keyup


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(new)

In [659]:
from itertools import groupby, count
from operator import itemgetter

def new_sort(df):
        
    df_2 = pd.DataFrame(columns=["key", "timestamp", "type"])
    indexes = []
    for i in range(len(df)):
        if i not in indexes:
            df_2 = df_2.append( df.loc[i,:])
            letter = df.loc[i,"key"]
            indexes.append(i)

            for j in range(i+1, len(df)):
                if ((df.loc[j,"key"] == df.loc[i,"key"]) and (j not in indexes)):

                    df_2 = df_2.append( df.loc[j,:])
                    indexes.append(j)
                    break;
                    
    return df_2.reset_index(drop=True)
    

def test_repeating_pattern(lst, pattern=("keydown", "keyup")):
    pat_len = len(pattern)
    assert "keydown" == lst[0], "keydown does not start the list: {}".format(lst[0])
    assert len(lst) % pat_len == 0, "mismatched length of list"
    assert list(pattern) * (len(lst) // pat_len) == lst, "the list does not follow the correct pattern"
    
def assess_repeating_key_compression_pattern(lst, pattern=("keydown", "keyup")):
    
    assert set(pattern).issubset(set(lst))
    pat_len = len(pattern)
    if ("keydown" == lst[0]) and (len(lst) % pat_len == 0) and (list(pattern) * (len(lst) // pat_len) == lst):
        return True
    else:
        return False

def combine_shift_keydowns_without_matching_keyup(df):
    """
    Function assumes that df has been sorted before getting this far.
    """
    
    # Get the index of all shift keydowns (these are the ones causing the registration problems)
    idxs = df.index[(df['key'] == 'shift') & (df['type'] == 'keydown')].tolist()
    
    # Locate all contiguous sub-sequences
    keydown_groups = []
    for k, g in groupby(enumerate(idxs), lambda ix: ix[0] - ix[1]):
        keydown_groups.append(list(map(itemgetter(1), g)))
        
    # Check what is inside shift groups (if they only contain 'keydown' or 'keyup' there is a problem)
    removal_keydown_coordinates = []
    for g in keydown_groups:
        if len(g) > 1:
            # Do this if the immediate key after each group is a "keyup"
            if df.loc[g[-1]+1, "type"] == 'keyup':
                removal_keydown_coordinates.extend(g[:-1])
            else:
                # Do this if there is no preceeding "keyup"
                removal_keydown_coordinates.extend(g)
        
    # In-place operation, no need to return anything. Cannot reset index at this point.
    df.drop(df.index[removal_keydown_coordinates], inplace=True)
    
    
def remove_superfluous_shifts(df):
    
    ### NOTE the backspace operator has to appear before this
    
    idxs = df.index[df['key'] == 'shift'].tolist()
    
    # Locate a contiguous sub-sequence at the start of the sentence
    shift_groups = []
    for k, g in groupby(enumerate(idxs), lambda ix: ix[0] - ix[1]):
        shift_groups.append(list(map(itemgetter(1), g)))
        
    # Check what is inside shift groups (if they only contain 'keydown' or 'keyup' there is a problem)
    removal_shift_coordinates = []
    for g in shift_groups:
        if set(g) == 'keydown' or set(g) == 'keydown':
            removal_shift_coordinates.append(g[:-1])
            
    
        
    if len(shift_groups[0]) > 2 and df.type[shift_groups[0][-1]] == 'keyup' and df.type[shift_groups[0][-2]] == 'keydown':
        # Coordinates to remove
        df.drop(df.index[shift_groups[0][:-2]], inplace=True)
        
    # Shifts should only ever appear as contiguous pairs
    if not all([len(x) == 2 for x in shift_groups[1:]]): 
        print("Other shift combinations are also longer than they should be.\n")
        print(shift_groups)
        
    return df

def lookup(v, d={}, c=count()):
    if v in d:
        return d.pop(v)
    else:
        d[v] = next(c)
        return d[v]


def reorder_key_timestamp_columns_mrc(df: pd.DataFrame):

    # Check that the column is of even length
    assert len(df) % 2 == 0, df.values

    # Use lookup function to extract the next row-order
    df["new_row_order"] = df.key.map(lookup)

    return df.sort_values(by="new_row_order", kind="mergesort").drop("new_row_order", axis=1).reset_index(drop=True)
